In [7]:
import pandas as pd
import ast
import re
import unidecode

UNITS = r"g|gr|grs|grams|gramos|kg|ml|mls|cl|l|cc|unitat|unitats|unit|u|dent|dents|manat|manats|cullerada|cullerades|blanc|blancs"

# Palabras que indican que es una unidad contable
COUNTABLE_UNITS = [
    'ou', 'ous',
    'all', 'alls',
    'llesca', 'llesques',
    'dent', 'dents',
    'cullerada', 'cullerades',
    'manat', 'manats',
    'blanc', 'blancs',
    'gra', 'grans',
    'tros', 'trossos',
    'tassa', 'tasses',
    'unitat', 'unitats'
]

def clean_parentheses(txt):
    """Elimina todo el contenido entre paréntesis y los paréntesis"""
    return re.sub(r'\([^)]*\)', '', txt).strip()

def clean_text(txt):
    """Limpia y normaliza el texto"""
    txt = clean_parentheses(txt)
    txt = unidecode.unidecode(txt.lower()).strip()
    txt = re.sub(r'\s+', ' ', txt)
    return txt.strip()

def parse_ingredient(ing):
    """Parser mejorado para ingredientes"""
    try:
        # Limpieza inicial
        txt = clean_text(ing)

        # 1. Ingredientes "al gust"
        if any(x in txt for x in ['al gust', 'al gusto', 'a gust']):
            nombre = txt.replace('al gust', '').replace('al gusto', '').replace('a gust', '').strip()
            return None, None, nombre

        # 2. Patrón básico: "X unidad_contable"
        # Ejemplo: "2 alls", "4 ous", "3 llesques de pernil"
        pattern_basic = r'^(\d+)\s+(' + '|'.join(COUNTABLE_UNITS) + r')\s*(?:de\s+|d\')?(.*)$'
        match = re.match(pattern_basic, txt)
        if match:
            cantidad, unidad, resto = match.groups()
            nombre = unidad if not resto else f"{unidad} de {resto}"
            return float(cantidad), None, nombre.strip()

        # 3. Patrón "X - Y g de ingrediente"
        pattern_range = r'^(\d+)\s*-\s*(\d+)\s*(g|gr|grs|grams|gramos|kg|ml|mls|cl|l)\s+(?:de\s+|d\')?(.+)$'
        match = re.match(pattern_range, txt)
        if match:
            _, cantidad, unidad, nombre = match.groups()  # tomamos el segundo número
            if unidad in ['gr', 'grs', 'grams', 'gramos']:
                unidad = 'g'
            return float(cantidad), unidad, nombre

        # 4. Patrón "XXg de ingrediente" o "XX g de ingrediente"
        pattern_weight = r'^(\d+(?:[.,]\d+)?)\s*(g|gr|grs|grams|gramos|kg|ml|mls|cl|l)\s+(?:de\s+|d\')?(.+)$'
        match = re.match(pattern_weight, txt)
        if match:
            cantidad, unidad, nombre = match.groups()
            cantidad = float(cantidad.replace(',', '.'))
            if unidad in ['gr', 'grs', 'grams', 'gramos']:
                unidad = 'g'
            return cantidad, unidad, nombre

        # 5. Patrón "ingrediente XXg"
        pattern_reverse = r'^(.+?)\s+(\d+(?:[.,]\d+)?)\s*(g|gr|grs|grams|gramos|kg|ml|mls|cl|l)$'
        match = re.match(pattern_reverse, txt)
        if match:
            nombre, cantidad, unidad = match.groups()
            cantidad = float(cantidad.replace(',', '.'))
            if unidad in ['gr', 'grs', 'grams', 'gramos']:
                unidad = 'g'
            return cantidad, unidad, nombre

        # Si no coincide con ningún patrón, devolver el texto limpio
        return None, None, txt

    except Exception as e:
        print(f"Error procesando '{ing}': {str(e)}")
        return None, None, txt

# Verificar casos específicos
ejemplos = [
    "2 alls",
    "4 ous",
    "3 llesques de pernil salat",
    "1 manat de julivert",
    "2 blancs d'ou",
    "150 - 200 g de bledes",
    "100g de patates",
    "tomàquet 200g",
    "1 cullerada de mantega",
    "2 dents d'all",
    "3 grans d'all",
    "2 tasses de llet"
]

print("\nVerificación de casos específicos:")
for ejemplo in ejemplos:
    cant, uni, nom = parse_ingredient(ejemplo)
    print(f"\nOriginal: {ejemplo}")
    print(f"Procesado: cantidad={cant}, unidad={uni}, nombre={nom}")

# Procesamiento del CSV
df = pd.read_csv("recetas_cat_0301.csv")

# Procesar ingredientes
rows = []
for idx, row in df.iterrows():
    receta = row["nom"]
    try:
        ingredientes = ast.literal_eval(row["ingredients"])
        for ing in ingredientes:
            cantidad, unidad, nombre = parse_ingredient(ing)
            rows.append({
                "receta": receta,
                "ingrediente_original": ing,
                "cantidad": cantidad,
                "unidad": unidad,
                "ingrediente_estandar": nombre
            })
    except Exception as e:
        print(f"Error en receta {receta}: {str(e)}")
        continue

# Crear DataFrame y mostrar resultados
ingredientes_df = pd.DataFrame(rows)

# Guardar resultados
ingredientes_df.to_csv("ingredientes_estandarizados_v10.csv", index=False)

# Exportar a CSV
ingredientes_df.to_csv("ingredientes_estandarizados.csv", index=False)


Verificación de casos específicos:

Original: 2 alls
Procesado: cantidad=2.0, unidad=None, nombre=all de s

Original: 4 ous
Procesado: cantidad=4.0, unidad=None, nombre=ou de s

Original: 3 llesques de pernil salat
Procesado: cantidad=3.0, unidad=None, nombre=llesques de pernil salat

Original: 1 manat de julivert
Procesado: cantidad=1.0, unidad=None, nombre=manat de julivert

Original: 2 blancs d'ou
Procesado: cantidad=2.0, unidad=None, nombre=blanc de s d'ou

Original: 150 - 200 g de bledes
Procesado: cantidad=200.0, unidad=g, nombre=bledes

Original: 100g de patates
Procesado: cantidad=100.0, unidad=g, nombre=patates

Original: tomàquet 200g
Procesado: cantidad=200.0, unidad=g, nombre=tomaquet

Original: 1 cullerada de mantega
Procesado: cantidad=1.0, unidad=None, nombre=cullerada de mantega

Original: 2 dents d'all
Procesado: cantidad=2.0, unidad=None, nombre=dent de s d'all

Original: 3 grans d'all
Procesado: cantidad=3.0, unidad=None, nombre=gra de ns d'all

Original: 2 tasses d

In [8]:
ingredientes_df.sample(40)

,receta,ingrediente_original,cantidad,unidad,ingrediente_estandar
53,Costelles de porc al romaní i boniato (prebiòtc),1⁄2 llimona,NaN,None,1/2 llimona
633,Pa (lli) amb oli amb pernil dolç/salat,Sal,NaN,None,sal
127,Amanida de sardines,100 g de pebre verd,100.0,g,pebre verd
527,Alvocat farcit de pollastre,5g de mostassa Dijon,5.0,g,mostassa dijon
323,Bombó de xocolata,10g d'eritritol,10.0,g,eritritol
544,Pasta de carabassí scampi,1-2 dents d'all,NaN,None,1-2 dents d'all
641,Crema de xampinyons i Fruits secs,1 fulla de llorer (opcional),NaN,None,1 fulla de llorer
870,"Amanida de ruca amb iogurt grec, ametlles i tahin",2 cullerada sopera de iogurt grec,2.0,None,cullerada de sopera de iogurt grec
453,Pitera de pollastre Cordon-bleu,10g de mantega,10.0,g,mantega
701,Crep amb xocolata i fraules,60 grams de formatge crema,60.0,None,gra de ms de formatge crema
